In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [26]:
#dir = './sequence/*'
dir = './latest_sequence/test/*'

In [27]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [7]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [8]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [9]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = 1

In [10]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    #loss2 = losses.kld(y_true, y_pred)
    return loss1# * 0.7 + loss2 * 0.3

In [17]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

In [18]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [11]:
model_path = 'model_save/mse_weights/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min')#, period=5)

In [29]:
#lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])
lstm_autoencoder.fit_generator(train_generator(all_data), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/500
1/1 [==============================] - 0s 25ms/step - loss: 1.9650
Epoch 2/500
1/1 [==============================] - 0s 13ms/step - loss: 1.3625
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 2.1616
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 900.8760
Epoch 5/500
1/1 [==============================] - 0s 10ms/step - loss: 1.9456
Epoch 6/500
1/1 [==============================] - 0s 9ms/step - loss: 3.0851
Epoch 7/500
1/1 [==============================] - 0s 10ms/step - loss: 3.3277
Epoch 8/500
1/1 [==============================] - 0s 11ms/step - loss: 4.7833
Epoch 9/500
1/1 [==============================] - 0s 12ms/step - loss: 841.8716
Epoch 10/500
1/1 [==============================] - 0s 8ms/step - loss: 6.0274
Epoch 11/500
1/1 [==============================] - 0s 12ms/step - loss: 7.7406
Epoch 12/500
1/1 [==============================] - 0s 9ms/step - loss: 7.9711
Epoch 13/500
1/1 [==============================] 

Epoch 103/500
1/1 [==============================] - 0s 11ms/step - loss: 16.9847
Epoch 104/500
1/1 [==============================] - 0s 8ms/step - loss: 746.4302
Epoch 105/500
1/1 [==============================] - 0s 10ms/step - loss: 16.4110
Epoch 106/500
1/1 [==============================] - 0s 10ms/step - loss: 17.2032
Epoch 107/500
1/1 [==============================] - 0s 8ms/step - loss: 16.5377
Epoch 108/500
1/1 [==============================] - 0s 9ms/step - loss: 17.1187
Epoch 109/500
1/1 [==============================] - 0s 8ms/step - loss: 745.8074
Epoch 110/500
1/1 [==============================] - 0s 9ms/step - loss: 16.5162
Epoch 111/500
1/1 [==============================] - 0s 8ms/step - loss: 17.3337
Epoch 112/500
1/1 [==============================] - 0s 10ms/step - loss: 16.6962
Epoch 113/500
1/1 [==============================] - 0s 10ms/step - loss: 17.2937
Epoch 114/500
1/1 [==============================] - 0s 9ms/step - loss: 744.3879
Epoch 115/500
1/1 [=

Epoch 204/500
1/1 [==============================] - 0s 8ms/step - loss: 732.2480
Epoch 205/500
1/1 [==============================] - 0s 9ms/step - loss: 17.9033
Epoch 206/500
1/1 [==============================] - 0s 9ms/step - loss: 18.6337
Epoch 207/500
1/1 [==============================] - 0s 9ms/step - loss: 17.9100
Epoch 208/500
1/1 [==============================] - 0s 8ms/step - loss: 18.4217
Epoch 209/500
1/1 [==============================] - 0s 9ms/step - loss: 732.9008
Epoch 210/500
1/1 [==============================] - 0s 8ms/step - loss: 17.7880
Epoch 211/500
1/1 [==============================] - 0s 9ms/step - loss: 18.6577
Epoch 212/500
1/1 [==============================] - 0s 9ms/step - loss: 18.0565
Epoch 213/500
1/1 [==============================] - 0s 8ms/step - loss: 18.6716
Epoch 214/500
1/1 [==============================] - 0s 9ms/step - loss: 730.8369
Epoch 215/500
1/1 [==============================] - 0s 8ms/step - loss: 18.0101
Epoch 216/500
1/1 [======

Epoch 305/500
1/1 [==============================] - 0s 9ms/step - loss: 16.1244
Epoch 306/500
1/1 [==============================] - 0s 9ms/step - loss: 17.2990
Epoch 307/500
1/1 [==============================] - 0s 12ms/step - loss: 16.9183
Epoch 308/500
1/1 [==============================] - 0s 11ms/step - loss: 17.6028
Epoch 309/500
1/1 [==============================] - 0s 9ms/step - loss: 720.2617
Epoch 310/500
1/1 [==============================] - 0s 10ms/step - loss: 16.7913
Epoch 311/500
1/1 [==============================] - 0s 8ms/step - loss: 16.7097
Epoch 312/500
1/1 [==============================] - 0s 10ms/step - loss: 15.0542
Epoch 313/500
1/1 [==============================] - 0s 8ms/step - loss: 15.6124
Epoch 314/500
1/1 [==============================] - 0s 10ms/step - loss: 729.9780
Epoch 315/500
1/1 [==============================] - 0s 10ms/step - loss: 14.9381
Epoch 316/500
1/1 [==============================] - 0s 7ms/step - loss: 15.7910
Epoch 317/500
1/1 [=

Epoch 406/500
1/1 [==============================] - 0s 11ms/step - loss: 12.9749
Epoch 407/500
1/1 [==============================] - 0s 12ms/step - loss: 12.1826
Epoch 408/500
1/1 [==============================] - 0s 8ms/step - loss: 12.6017
Epoch 409/500
1/1 [==============================] - 0s 10ms/step - loss: 713.9961
Epoch 410/500
1/1 [==============================] - 0s 8ms/step - loss: 11.6610
Epoch 411/500
1/1 [==============================] - 0s 8ms/step - loss: 12.1108
Epoch 412/500
1/1 [==============================] - 0s 8ms/step - loss: 11.1892
Epoch 413/500
1/1 [==============================] - 0s 9ms/step - loss: 11.6254
Epoch 414/500
1/1 [==============================] - 0s 8ms/step - loss: 710.7634
Epoch 415/500
1/1 [==============================] - 0s 10ms/step - loss: 10.5167
Epoch 416/500
1/1 [==============================] - 0s 10ms/step - loss: 10.7463
Epoch 417/500
1/1 [==============================] - 0s 8ms/step - loss: 9.6399
Epoch 418/500
1/1 [===

In [13]:
model_json = lstm_autoencoder.to_json()
filename = 'last_mse_lstmae' # input('filename: ') #
with open('model_save/mse_weights/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model_save/mse_weights/weights_' +  filename + '.h5')

In [12]:
filename = "last_mse_lstmae"
loaded_model = model_from_json(open('model_save/mse_weights/' +filename + '.json').read())
loaded_model.load_weights('model_save/mse_weights/weights_' + filename + '.h5')

W1010 15:48:40.110543 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 15:48:40.120297 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1010 15:48:40.123097 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1010 15:48:40.688915 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [30]:
loaded_model = lstm_autoencoder

In [21]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

NameError: name 'x_test' is not defined

In [24]:
encoder = Model(loaded_model.input, loaded_model.layers[2].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [32]:
def euclidean_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.sqrt(np.sum(a-b) ** 2)

In [33]:
predicted = encoder.predict(all_data)

In [35]:
obj = predicted[1]

In [36]:
for sbj in predicted:
    print(euclidean_distance(obj, sbj))

0.025938525795936584
0.0
0.03398488461971283
2.0210580825805664
0.006183251738548279
